import statements

In [4]:
import numpy as np
import pandas as pd

Function to calculate correctness rate

The Correcness rate represents the proportion of correct answers given by a user over their total attempts.

We calculate it by 
 - For each user, maintain a list of all their answers (correct or incorrect)
 - For each attempt, caculate the correctness rate as the ratio of correct answers to the total number of attempts up to that point.

In [5]:
def calculate_correctness_rate(user_attempts):
    correct_answers = sum(user_attempts)
    total_attempts = len(user_attempts)
    return correct_answers / total_attempts if total_attempts>0 else 0

Function to calculate confidence level
The confidence level is a mesaure of how confident a user might be in their answer, based on their performance and behavior (eg. marking the questions for review)

We calculate it by 
 - starting with a base confidence level of 3
 - for correct answer increase confidence +1 if first attempt, another +1 if question is not marked for review (max 5)
 - for incorrect answer decrease confidence -1 if the answer is marked for review,  another -1 for additional attempt beyond first (min 1)

In [6]:
def calculate_confidence_level(row):

    if row['answer'] == 1:
        if(row['time_taken']<60):
            ttC = 1
        else:
            ttC = 0
        return min(5,3 + (row['attempt_number'] < 3) + (not row['marked_for_review']) + ttC)
    else:
        if(row['time_taken']<60):
            ttC = 0
        else:
            ttC = 1
        return max(1,3 - (row['marked_for_review']) - row['attempt_number']-ttC)


Function to determine the label based on the defined rules i.e
- attempt_number
- correctness_rate
- marked_for_review
- confidence_level
- answer

Repeat the same question (label = 1) if
 - user has attempted the question multiple times (high `attempt_number`)
 - correctness rate is low
 - user marked the question for review
 - confidence level is low
 - answer is incorrect
 i.e
 attempt number > 1 and correctness rate < 0.7
 marked for review == 1
 confidence level < 3
 answer incorrect i.e == 0

else 

label = 0
Move on from the question (label = 0) if
- The user answered the question correctly on the first or second attempt
- the correctness rate is high
- the user did not mark the question for review 
- the confidence level is high

In [7]:
def determine_label(row):
    if(row['attempt_number'] > 2 and row['correctness_rate'] < 0.7) or row['confidence_level'] < 3 or row['answer'] == 0:
        return 1 #repeat the same question
    else:
        return 0 #move on from the question


Now generating a synthetic dataset (dummy data)

In [8]:
np.random.seed(42)
QuestionBank = pd.read_csv('questionBank.csv')

qb_dbms_df = QuestionBank[QuestionBank['domain']=='DBMS']
qb_dsa_df = QuestionBank[QuestionBank['domain']=='DSA']

domains = ["DBMS","Data Structures"]
subdomains_dbms = list(qb_dbms_df['subdomain'].unique())
subdomains_ds = list(qb_dsa_df['subdomain'].unique())
subdomains = subdomains_dbms + subdomains_ds

#initialize an empty list to collect rows

data = []


#generate data for users and questions

for user_id in range(1,9): # 5 users
    user_attempts = []
    for question_id in range(100,400): # 300 questions
        domain = np.random.choice(domains)
        subdomain = np.random.choice(subdomains_dbms if domain == "DBMS" else subdomains_ds)
        for attempt_number in range(1,np.random.randint(2,5)): # 1 to 3 attempts for questions
            time_taken = np.random.randint(30,120) #20 to 120 seconds
            answer = np.random.randint(0,2) # 0 or 1 (incorrect or correct)
            marked_for_review = np.random.randint(0,2) # 0 or 1 (unmarked or marked)

            #update user attempts and calculate correctness rate

            user_attempts.append(answer)
            correctness_rate = calculate_correctness_rate(user_attempts)

            #Calculate teh confidence level

            confidence_level = calculate_confidence_level({
                'answer':answer,
                'attempt_number':attempt_number,
                'marked_for_review':marked_for_review,
                'time_taken':time_taken
            })

            # Determine the label

            label = determine_label({
                'attempt_number' : attempt_number,
                'correctness_rate':correctness_rate,
                'marked_for_review':marked_for_review,
                'confidence_level' : confidence_level,
                'answer' : answer
            })

            #append the row to the Dataframe

            row ={
                'user_id':user_id,
                'question_id':question_id,
                'domain':domain,
                'subdomain':subdomain,
                'attempt_number':attempt_number,
                'time_taken':time_taken,
                'answer':answer,
                'marked_for_review':marked_for_review,
                'correctness_rate':correctness_rate,
                'confidence_level':confidence_level,
                'label':label
            }
            data.append(row)

#check the dataset now
df = pd.DataFrame(data)
# print(df.head())

df.to_csv('dummyData.csv',index = False)

Now
Weighted KNN

We need to prepare the data (train test split)
train the model
evaluate the model


Importing the sklearn stuff

Load the dataset

Prepare dataset and split 

Initialize the weighted KNN model, train the model and make predictions

Evalutate the model now

test

The next one seems to work


In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Load the dataset
df = pd.read_csv('dummyData.csv')

# Prepare the feature set and labels
X = df.drop(columns=['label', 'user_id', 'question_id', 'domain', 'subdomain'])
y = df['label']

# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the weighted KNN model
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn.fit(X_train_scaled, y_train)

# New input data
new_data = pd.DataFrame({
    'attempt_number': [1],
    'time_taken': [85],
    'answer': [1],
    'marked_for_review': [1],
    'correctness_rate': [0.7],
    'confidence_level': [3]
})

# Scale the new data using the same scaler
new_data_scaled = scaler.transform(new_data)

# Predict using the model
prediction = knn.predict(new_data_scaled)

# Print the result
print(f'Predicted label for the new input: {prediction[0]}')


Predicted label for the new input: 0


In [10]:
def assign_category(row):
    # Extract the percentage of repeats
    percent_repeat = row['percent_repeat']
    
    # Define thresholds for categorization
    if percent_repeat > 60:
        return 2
    elif percent_repeat < 50:
        return 0
    else:
        return 1


In [11]:
df = pd.read_csv('dummyData.csv')

# Select the specified columns
selected_columns = ['user_id', 'question_id', 'domain', 'subdomain', 'label']
df_selected = df[selected_columns]

# Print the selected DataFrame
# print(df_selected)

# Calculate percentages for each subdomain
subdomain_stats = df_selected.groupby('subdomain')['label'].agg(['mean', 'count']).reset_index()
subdomain_stats['percent_repeat'] = subdomain_stats['mean'] * 100

subdomain_stats['category'] = subdomain_stats.apply(assign_category, axis=1)

print(subdomain_stats)

# The new dataset
classified_data = subdomain_stats[['subdomain', 'category']]

print(classified_data)


            subdomain      mean  count  percent_repeat  category
0           Algorithm  0.518325    191       51.832461         1
1          Algorithms  0.573394    218       57.339450         1
2               Array  0.511450    131       51.145038         1
3   Backup & Recovery  0.547170    159       54.716981         1
4     Database Design  0.497175    177       49.717514         0
5     Database Models  0.642857    168       64.285714         2
6               Graph  0.579545    176       57.954545         1
7             Hashing  0.594340    212       59.433962         1
8                Heap  0.588745    231       58.874459         1
9            Indexing  0.547619    168       54.761905         1
10              Joins  0.574194    155       57.419355         1
11               Keys  0.591623    191       59.162304         1
12        Linked List  0.560870    230       56.086957         1
13      Normalization  0.560847    189       56.084656         1
14              Queue  0.

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

# Encoding the categories
label_encoder = LabelEncoder()
classified_data.loc[:,'category_encoded'] = label_encoder.fit_transform(classified_data['category'])
print (classified_data)
# Features and labels
X = subdomain_stats[['percent_repeat']]
y = classified_data['category_encoded']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=47)

# Train Weighted KNN
knn_model = KNeighborsClassifier(weights='distance')
knn_model.fit(X_train, y_train)

# Evaluate the models

knn_accuracy = knn_model.score(X_test, y_test)


print(f'\nWeighted KNN Accuracy: {knn_accuracy}')


            subdomain  category  category_encoded
0           Algorithm         1                 1
1          Algorithms         1                 1
2               Array         1                 1
3   Backup & Recovery         1                 1
4     Database Design         0                 0
5     Database Models         2                 2
6               Graph         1                 1
7             Hashing         1                 1
8                Heap         1                 1
9            Indexing         1                 1
10              Joins         1                 1
11               Keys         1                 1
12        Linked List         1                 1
13      Normalization         1                 1
14              Queue         1                 1
15         SQL Basics         1                 1
16    SQL Constraints         2                 2
17      SQL Functions         2                 2
18          Searching         1                 1


C:\Users\devansh\AppData\Local\Temp\ipykernel_15372\2932094078.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classified_data.loc[:,'category_encoded'] = label_encoder.fit_transform(classified_data['category'])


# GENERATING A NEW QUESTION PAPER

First creating a realistic data set on which we will run the models

In [13]:
data = [
    [1, 122, 'DBMS', 'SQL Basics', [1,1,1,1,1], 40, 1, 1],
    [1, 133, 'DBMS', 'Normalization', [0, 0, 1, 0], 85, 0, 0],
    [1, 144, 'DBMS', 'SQL Constraints', [1], 75, 1, 1],
    [1, 155, 'DBMS', 'Keys', [1,1,1,1,1], 90, 0, 0],
    [1, 166, 'DBMS', 'SQL Functions', [0, 0, 1, 0], 55, 1, 1],
    [1, 177, 'DBMS', 'Joins', [1], 100, 1, 0],
    [1, 188, 'DBMS', 'Stored Procedures', [1, 0, 1], 110, 0, 1],
    [1, 199, 'DBMS', 'Transactions', [1,1,1,1,1], 65, 1, 0],
    [1, 110, 'DBMS', 'Database Models', [1], 95, 0, 1],
    [1, 121, 'DBMS', 'Indexing', [1,1,1,1,1], 70, 1, 0],
    [1, 132, 'DBMS', 'Views', [1, 0, 1], 85, 0, 0],
    [1, 143, 'DBMS', 'Database Design', [1], 100, 1, 1],
    [1, 154, 'DBMS', 'Security', [0, 0, 1], 65, 0, 0],
    [1, 165, 'DBMS', 'Backup & Recovery', [1], 105, 1, 1],
    [1, 176, 'DSA', 'Searching', [0, 0, 1], 35, 0, 1],
    [1, 187, 'DSA', 'Stack', [1], 115, 1, 0],
    [1, 198, 'DSA', 'Sorting', [1, 0], 90, 0, 0],
    [1, 109, 'DSA', 'Queue', [0, 0, 1], 50, 1, 1],
    [1, 120, 'DSA', 'Graph', [1], 85, 0, 0],
    [1, 131, 'DSA', 'Heap', [1,1,1], 60, 1, 0],
    [1, 142, 'DSA', 'Tree', [1], 125, 0, 1],
    [1, 153, 'DSA', 'Algorithms', [1, 0], 45, 1, 0],
    [1, 164, 'DSA', 'Hashing', [1,1,1], 75, 0, 1],
    [1, 175, 'DSA', 'Linked List', [1], 55, 1, 0],
    [1, 186, 'DSA', 'Array', [1,1,1], 90, 0, 1],
    [1, 197, 'DSA', 'Algorithm', [1, 0], 80, 1, 0],
    [1, 108, 'DBMS', 'SQL Basics', [1, 0], 55, 1, 0],
    [1, 119, 'DBMS', 'Normalization', [1], 70, 0, 1],
    [1, 130, 'DBMS', 'SQL Constraints', [1,1,1], 60, 1, 0],
    [1, 141, 'DBMS', 'Keys', [1, 0], 110, 0, 1],
    [1, 152, 'DBMS', 'SQL Functions', [1], 50, 1, 0],
    [1, 163, 'DBMS', 'Joins', [0, 0, 1], 95, 1, 0],
    [1, 174, 'DBMS', 'Stored Procedures', [1], 80, 0, 1],
    [1, 185, 'DBMS', 'Transactions', [1, 0], 120, 1, 0],
    [1, 196, 'DBMS', 'Database Models', [0, 0, 1], 65, 0, 0],
    [1, 107, 'DBMS', 'Indexing', [1], 105, 1, 1],
    [1, 118, 'DBMS', 'Views', [1, 0], 70, 0, 0],
    [1, 129, 'DBMS', 'Database Design', [0, 0, 1], 130, 1, 1],
    [1, 140, 'DBMS', 'Security', [1], 60, 0, 0],
    [1, 151, 'DBMS', 'Backup & Recovery', [1, 0], 100, 1, 1],
    [1, 162, 'DSA', 'Searching', [0, 0, 1], 35, 0, 1],
    [1, 173, 'DSA', 'Stack', [1], 110, 1, 0],
    [1, 184, 'DSA', 'Sorting', [1, 0], 85, 0, 0],
    [1, 195, 'DSA', 'Queue', [0, 0, 1], 50, 1, 1],
    [1, 106, 'DSA', 'Graph', [1], 95, 0, 0],
    [1, 117, 'DSA', 'Heap', [1, 0, 1], 60, 1, 0],
    [1, 128, 'DSA', 'Tree', [1, 0], 120, 0, 1],
    [1, 139, 'DSA', 'Algorithms', [1, 0, 1], 45, 1, 0],
    [1, 150, 'DSA', 'Hashing', [1, 0], 75, 0, 1],
    [1, 161, 'DSA', 'Linked List', [1, 0, 1], 55, 1, 0],
    [1, 172, 'DSA', 'Array', [1], 90, 0, 1],
    [1, 183, 'DSA', 'Algorithm', [1, 0, 1], 80, 1, 0],
    [1, 201, 'DBMS', 'SQL Basics', [1, 0], 60, 1, 0],
    [1, 202, 'DBMS', 'Normalization', [1], 70, 0, 1],
    [1, 203, 'DBMS', 'SQL Constraints', [1, 0, 1], 65, 1, 0],
    [1, 204, 'DBMS', 'Keys', [1, 0], 80, 0, 1],
    [1, 205, 'DBMS', 'SQL Functions', [1], 75, 1, 0],
    [1, 206, 'DBMS', 'Joins', [1, 0,1,1], 90, 1, 0],
    [1, 207, 'DBMS', 'Stored Procedures', [1, 0, 1], 100, 0, 1],
    [1, 208, 'DBMS', 'Transactions', [1], 85, 1, 0],
    [1, 209, 'DBMS', 'Database Models', [1, 0,1,1], 95, 0, 1],
    [1, 210, 'DBMS', 'Indexing', [1], 110, 1, 0],
    [1, 211, 'DBMS', 'Views', [1, 0,1,1], 80, 0, 1],
    [1, 212, 'DBMS', 'Database Design', [0], 105, 1, 1],
    [1, 213, 'DBMS', 'Security', [1, 0, 1], 90, 0, 0],
    [1, 214, 'DBMS', 'Backup & Recovery', [0], 115, 1, 1],
    [1, 215, 'DSA', 'Searching', [1, 0,1,1], 45, 0, 1],
    [1, 216, 'DSA', 'Stack', [1, 1, 1, 1], 100, 1, 0],
    [1, 217, 'DSA', 'Sorting', [0], 80, 0, 1],
    [1, 218, 'DSA', 'Queue', [1, 0], 60, 1, 1],
    [1, 219, 'DSA', 'Graph', [1, 1, 1, 1], 90, 0, 0],
    [1, 220, 'DSA', 'Heap', [0], 70, 1, 0],
    [1, 221, 'DSA', 'Tree', [1, 1, 1, 1], 125, 0, 1],
    [1, 222, 'DSA', 'Algorithms', [0], 55, 1, 0],
    [1, 223, 'DSA', 'Hashing', [1, 0], 85, 0, 1],
    [1, 224, 'DSA', 'Linked List', [1, 1, 1, 1], 60, 1, 0],
    [1, 225, 'DSA', 'Array', [0], 95, 0, 1],
    [1, 226, 'DSA', 'Algorithm', [1, 0], 80, 1, 0],
    [1, 227, 'DBMS', 'SQL Basics', [1, 1, 1, 1], 50, 1, 1],
    [1, 228, 'DBMS', 'Normalization', [0], 65, 0, 0],
    [1, 229, 'DBMS', 'SQL Constraints', [1, 0], 70, 1, 1],
    [1, 230, 'DBMS', 'Keys', [1, 0, 1], 85, 0, 1],
    [1, 231, 'DBMS', 'SQL Functions', [0], 90, 1, 0],
    [1, 232, 'DBMS', 'Joins', [1, 0], 80, 1, 0],
    [1, 233, 'DBMS', 'Stored Procedures', [1], 95, 0, 1],
    [1, 234, 'DBMS', 'Transactions', [1, 0, 1], 75, 1, 0],
    [1, 235, 'DBMS', 'Database Models', [1], 110, 0, 1],
    [1, 236, 'DBMS', 'Indexing', [1, 0], 100, 1, 0],
    [1, 237, 'DBMS', 'Views', [1], 85, 0, 1],
    [1, 238, 'DBMS', 'Database Design', [1, 0], 105, 1, 1],
    [1, 239, 'DBMS', 'Security', [1], 95, 0, 0],
    [1, 240, 'DBMS', 'Backup & Recovery', [1, 0, 1], 115, 1, 1],
    [1, 241, 'DSA', 'Searching', [1], 50, 0, 1],
    [1, 242, 'DSA', 'Stack', [1, 0], 120, 1, 0],
    [1, 243, 'DSA', 'Sorting', [1, 0, 1], 85, 0, 1],
    [1, 244, 'DSA', 'Queue', [1], 65, 1, 1],
    [1, 245, 'DSA', 'Graph', [1, 0], 90, 0, 0],
    [1, 246, 'DSA', 'Heap', [1, 0, 1], 70, 1, 0],
    [1, 247, 'DSA', 'Tree', [1], 125, 0, 1],
    [1, 248, 'DSA', 'Algorithms', [1, 0], 55, 1, 0],
    [1, 249, 'DSA', 'Hashing', [1], 85, 0, 1],
    [1, 250, 'DSA', 'Linked List', [1, 0], 60, 1, 0],
    [1, 251, 'DSA', 'Array', [0, 0, 1, 0], 95, 0, 1],
    [1, 252, 'DSA', 'Algorithm', [1], 80, 1, 0],
    [1, 253, 'DBMS', 'SQL Basics', [1, 0], 55, 1, 1],
    [1, 254, 'DBMS', 'Normalization', [1], 75, 0, 1],
    [1, 255, 'DBMS', 'SQL Constraints', [0, 0, 1, 0], 70, 1, 0],
    [1, 256, 'DBMS', 'Keys', [1], 85, 0, 1],
    [1, 257, 'DBMS', 'SQL Functions', [1, 0], 80, 1, 0],
    [1, 258, 'DBMS', 'Joins', [1], 90, 1, 0],
    [1, 259, 'DBMS', 'Stored Procedures', [1, 0], 100, 0, 1],
    [1, 260, 'DBMS', 'Transactions', [1], 65, 1, 0],
    [1, 261, 'DBMS', 'Database Models', [0, 0, 1, 0], 95, 0, 1],
    [1, 262, 'DBMS', 'Indexing', [1], 105, 1, 0],
    [1, 263, 'DBMS', 'Views', [1, 0], 80, 0, 1],
    [1, 264, 'DBMS', 'Database Design', [1], 110, 1, 1],
    [1, 265, 'DBMS', 'Security', [1, 0], 90, 0, 0],
    [1, 266, 'DBMS', 'Backup & Recovery', [1], 115, 1, 1],
    [1, 267, 'DSA', 'Searching', [1, 0], 45, 0, 1],
    [1, 268, 'DSA', 'Stack', [1], 120, 1, 0]
]



df_data_list = []

for i in range(len(data)):
    attCnt = len(data[i][4])
    correctness_rate = calculate_correctness_rate(data[i][4])
    confidence_level = calculate_confidence_level({
        'answer':data[i][6],
        'attempt_number':attCnt,
        'marked_for_review':data[i][7],
        'time_taken':data[i][5]
    })
    
    row = {
        'user_id':data[i][0],
        'question_id':data[i][1],
        'domain':data[i][2],
        'subdomain':data[i][3],
        'attempt_number':attCnt,
        'time_taken':data[i][5],
        'answer':data[i][6],
        'marked_for_review':data[i][7],
        'correctness_rate':correctness_rate,
        'confidence_level':confidence_level,
    }

    df_data_list.append(row)

dataFramePrimary = pd.DataFrame(df_data_list)

#now to get the data to send to model

inpData = dataFramePrimary.drop(columns=['user_id','question_id','domain','subdomain'])

# print(inpData)
#scale the above data and then make predictions and store them in original df
inpDataScale = scaler.transform(inpData)

# print(type(inpDataScale))

predict=knn.predict(inpDataScale)

QuestionLabelledData = []

for index,rows in inpData.iterrows():
    
    nRow = {
        'user_id':df_data_list[index].get('user_id'),
        'question_id':df_data_list[index].get('question_id'),
        'domain':df_data_list[index].get('domain'),
        'subdomain':df_data_list[index].get('subdomain'),
        'attempt_number':df_data_list[index].get('attempt_number'),
        'time_taken':df_data_list[index].get('time_taken'),
        'answer':df_data_list[index].get('answer'),
        'marked_for_review':df_data_list[index].get('marked_for_review'),
        'correctness_rate':df_data_list[index].get('correctness_rate'),
        'confidence_level':df_data_list[index].get('confidence_level'),
        'label':predict[index]
    }
    QuestionLabelledData.append(nRow)

questionLabeledDataFrame = pd.DataFrame(QuestionLabelledData)

#now classifying the topics
#selecting the required columns for secondary classification

toLabelTopicDataFrame = questionLabeledDataFrame[selected_columns]

#calculating percentages for each subdomain

tolabelTopic_stats = toLabelTopicDataFrame.groupby('subdomain')['label'].agg(['mean','count']).reset_index()

tolabelTopic_stats['percent_repeat'] = tolabelTopic_stats['mean'] * 100
print(tolabelTopic_stats)
secondth = knn_model.predict(tolabelTopic_stats.drop(columns=['subdomain','mean','count']))

topic_classification = []

def Tclassify(lab):
    if(lab == 0):
        return 'Move on from the Topic'
    elif(lab==1):
        return 'Repeat Same Topic with Less Weightage'
    elif(lab==2):
        return 'Repeat Same Topic with More Weightage'
    else:
        return 'Invalid'

for i in range(len(secondth)):
    newRow = {
        'subdomain':tolabelTopic_stats.get('subdomain')[i],
        'category':Tclassify(secondth[i]),
        'category_encoded':secondth[i]
    }
    topic_classification.append(newRow)

finalClassificationDataFrame = pd.DataFrame(topic_classification)

print(finalClassificationDataFrame)



            subdomain  mean  count  percent_repeat
0           Algorithm  0.25      4            25.0
1          Algorithms  0.50      4            50.0
2               Array  0.50      4            50.0
3   Backup & Recovery  0.40      5            40.0
4     Database Design  0.40      5            40.0
5     Database Models  0.60      5            60.0
6               Graph  0.25      4            25.0
7             Hashing  0.50      4            50.0
8                Heap  0.75      4            75.0
9            Indexing  0.00      5             0.0
10              Joins  0.20      5            20.0
11               Keys  0.60      5            60.0
12        Linked List  0.25      4            25.0
13      Normalization  0.40      5            40.0
14              Queue  0.50      4            50.0
15         SQL Basics  0.00      5             0.0
16    SQL Constraints  0.40      5            40.0
17      SQL Functions  0.40      5            40.0
18          Searching  0.80    

In [ ]:
#now to generate new question paper

#we need to extract the domains and subdomains the question paper needs first
#say the question paper has a limit of 20 questions across the domains 
#so 2 domains, 10 subdomains across them and 2 questions each??
#but we also need to increase weightage and decrease weightage
#so that means we should increase the number of questions so that we can manage the weightage

# so we need to determine how the weightage of domains will affect the paper layout



#first deciding the default layout and then according to the weightage we will add the questions 
#the thing we need to do is if more weightage then increase the amount of difficult questions as well

#for the subdomain if the classification is ____ then : 
#so more weightage : 5 questions
#less weightage : 3 questions
#move on : 1 question

#Generating a default question paper
#select the domain
#select the subdomains

In [17]:
import pandas as pd
import random

# Load the question bank dataset
QuestionBank = pd.read_csv('questionBank.csv')

# List of selected subdomains
selected_subdomain = [
    'SQL Basics',
    'Normalization',
    'SQL Constraints',
    'Keys',
    'SQL Functions',
    'Joins',
    'Transactions',
    'Security',
    'Backup & Recovery'
]

# Mapping the difficulty for each subdomain with a numeric value
difficultyMap = {'Easy': 1, 'Medium': 2, 'Hard': 3}
QuestionBank['difficulty_numeric'] = QuestionBank['difficulty'].map(difficultyMap)

# Filter questions by selected subdomains
subdomainQB = QuestionBank[QuestionBank['subdomain'].isin(selected_subdomain)]

# Calculate the average difficulty for each subdomain
subdomainQB_difficulty = subdomainQB.groupby('subdomain')['difficulty_numeric'].mean().reset_index()
subdomainQB_difficulty.columns = ['subdomain', 'avg_difficulty']

# Initialize weights dynamically
weights = dict.fromkeys(selected_subdomain, 0)

# Function to adjust weights
def adjust_weights(weights, subdomain, adjustment):
    if subdomain in weights:
        weights[subdomain] += adjustment
        if weights[subdomain] < -1:
            weights[subdomain] = -1
        elif weights[subdomain] > 1:
            weights[subdomain] = 1
    return weights



# Add weight column to DataFrame
subdomainQB_difficulty['weight'] = subdomainQB_difficulty['subdomain'].map(weights)

# Calculate the number of questions to select from each subdomain
min_questions = 1
total_questions = random.randint(20, 26)
additional_questions = total_questions - min_questions * len(selected_subdomain)

# Adjust question calculation to include weights
subdomainQB_difficulty['weighted_difficulty'] = (
    (subdomainQB_difficulty['avg_difficulty'] + subdomainQB_difficulty['weight']) / 
    (subdomainQB_difficulty['avg_difficulty'].max() + 1)
)

subdomainQB_difficulty['questions'] = min_questions + (
    (subdomainQB_difficulty['weighted_difficulty'] / subdomainQB_difficulty['weighted_difficulty'].sum()) * additional_questions
).astype(int)

# Adjust to ensure the total number of questions is within the required range
if subdomainQB_difficulty['questions'].sum() > total_questions:
    while subdomainQB_difficulty['questions'].sum() > total_questions:
        max_index = subdomainQB_difficulty['questions'].idxmax()
        subdomainQB_difficulty.at[max_index, 'questions'] -= 1
elif subdomainQB_difficulty['questions'].sum() < total_questions:
    while subdomainQB_difficulty['questions'].sum() < total_questions:
        min_index = subdomainQB_difficulty['questions'].idxmin()
        subdomainQB_difficulty.at[min_index, 'questions'] += 1
print(subdomainQB_difficulty)
# Ensure questions include all three difficulties and prioritize hard questions
selected_QIDS = []
for _, row in subdomainQB_difficulty.iterrows():
    subdomain = row['subdomain']
    num_questions = row['questions']

    subdomain_questions = subdomainQB[subdomainQB['subdomain'] == subdomain]

    easy_questions = subdomain_questions[subdomain_questions['difficulty'] == 'Easy']
    medium_questions = subdomain_questions[subdomain_questions['difficulty'] == 'Medium']
    hard_questions = subdomain_questions[subdomain_questions['difficulty'] == 'Hard']

    num_hard = min(num_questions // 2, len(hard_questions))
    num_easy_medium = num_questions - num_hard

    num_easy = min(num_easy_medium // 2, len(easy_questions))
    num_medium = num_easy_medium - num_easy

    if num_easy + num_medium + num_hard < num_questions:
        remaining = num_questions - (num_easy + num_medium + num_hard)
        num_hard += remaining

    # Now randomly selecting questions, ensuring they don't repeat frequently
    selected_QIDS += random.sample(list(hard_questions['qID']), min(num_hard, len(hard_questions)))
    selected_QIDS += random.sample(list(medium_questions['qID']), min(num_medium, len(medium_questions)))
    selected_QIDS += random.sample(list(easy_questions['qID']), min(num_easy, len(easy_questions)))        

# Adjust the number of selected questions to ensure it falls within the required range
while len(selected_QIDS) < 20:
    for _, row in subdomainQB_difficulty.iterrows():
        subdomain = row['subdomain']
        subdomain_questions = subdomainQB[subdomainQB['subdomain'] == subdomain]
        remaining_questions = subdomain_questions[~subdomain_questions['qID'].isin(selected_QIDS)]
        if len(remaining_questions) > 0:
            selected_QIDS.append(remaining_questions.sample(1)['qID'].values[0])
        if len(selected_QIDS) >= 20:
            break

while len(selected_QIDS) > 26:
    selected_QIDS.pop()

# Shuffle the selected qIDs to ensure a different order each run
random.shuffle(selected_QIDS)

print(selected_QIDS)


           subdomain  avg_difficulty  weight  weighted_difficulty  questions
0  Backup & Recovery        2.000000       0             0.625000          3
1              Joins        2.000000       0             0.625000          3
2               Keys        1.714286       0             0.535714          3
3      Normalization        2.000000       0             0.625000          3
4         SQL Basics        1.481481       0             0.462963          3
5    SQL Constraints        1.000000       0             0.312500          3
6      SQL Functions        1.346154       0             0.420673          2
7           Security        2.000000       0             0.625000          3
8       Transactions        2.200000       0             0.687500          3
[266, 195, 255, 253, 276, 257, 265, 180, 240, 218, 192, 178, 259, 196, 186, 242, 251, 308, 216, 203, 171, 299]
